In [2]:
import re
import unicodedata

import numpy as np
import pandas as pd

from tqdm import tqdm

In [3]:
df_train = pd.read_csv("../datasets/train_essays.csv")
train_essays = df_train["text"].to_list()
df_persuade = pd.read_csv("../datasets/persuade/persuade.csv")
persuade_essays = df_persuade["text"].to_list()

In [3]:
llm_essays = df_train[df_train["generated"] == 1]["text"].to_list()
for e in llm_essays:
    print(e + "\n\n")

This essay will analyze, discuss and prove one reason in favor of keeping the Electoral College in the United States for its presidential elections. One of the reasons to keep the electoral college is that it is better for smaller, more rural states to have more influence as opposed to larger metropolitan areas that have large populations. The electors from these states are granted two votes each. Those from larger, more populated areas are granted just one vote each. Smaller states tend to hold significant power because their two votes for president and vice president add up more than the votes of larger states that have many electors. This is because of the split of the electoral votes. Some argue that electors are not bound to vote for the candidate who won the most votes nationally. They do not have to vote for their own state's nominee unless their state has a winner take all system. However, there are states that have adopted laws that force their electors to vote for their state

In [4]:
def char_diffs(text1, text2):
    t = set(list("".join(text1)))
    p = set(list("".join(text2)))
    chars_to_remove = "".join([x for x in p if x not in t])
    return chars_to_remove

In [5]:
def clean_essay(text):
    # normalize text
    text = unicodedata.normalize("NFKD", text)
    text = text.encode("ascii", "ignore").decode("ascii")
    # text = text.replace("''", '"')
    # remove citations
    m = re.search(
        r"\n(Reference|References|Work Cited|Works Cited)", text, flags=re.IGNORECASE
    )
    if m:
        text = text[: m.start()]
    return text.strip()

In [6]:
persuade_essays = [clean_essay(x) for x in persuade_essays]
chars_to_remove = char_diffs(train_essays, persuade_essays)
print(chars_to_remove)

[(>\*`^);=$+~/#@|-]{}<


In [7]:
bad_essay_count = 0
for i, e in enumerate(persuade_essays):
    if any(x in e for x in chars_to_remove):
        bad_essay_count += 1
print(bad_essay_count)

13519


In [8]:
for c in chars_to_remove:
    counter = 0
    for e in persuade_essays:
        if c in e:
            counter += 1
    print("{}: {}".format(c, counter))

[: 349
(: 3391
>: 11
\: 39
*: 35
`: 89
^: 1
): 3419
;: 3487
=: 43
$: 707
+: 131
~: 14
/: 2319
#: 97
@: 6
|: 9
-: 9473
]: 335
{: 14
}: 14
<: 13


### Check PII tags

In [11]:
PII_tags = [
    "EMAIL_ADDRESS",
    "PHONE_NUMBER",
    "GENERIC_NAME",
    "PROPER_NAME",
    "LOCATION_NAME",
    "STATE_NAME",
    "CITY_STATE",
    "SCHOOL_NAME",
    "DAY_MONTH_YEAR",
    "MONTH_DAY_YEAR",
]

counter = 0
for e in persuade_essays:
    if any(x in e for x in PII_tags):
        counter += 1
        print(e)
        print("\n\n" + "=" * 80 + "\n\n")
print(f"{counter} out of {len(persuade_essays)} essays contain PII tags")

Nice day to go out with your friends enjoy and yourself, after a stressful week at work. You and your friends decided it will be a good idea to head to a soccer game to cheer your favorite team on.

Y'all give each other a task to bring something to feel comfortable doing the game. Some one bring chairs, snacks/food, and something to drink. You get in your car headed to the game, you at the red light when PROPER_NAME decided to text you" Make sure you got the drinks" light changes you start back driving focused on the text. Out of nowhere Errrrrrrr BOOM!.

Its really not a good idea to text don't matter the cause. You should pay attention at all times because you don't know what could happen. You could injury yourself or even kill someone.

No one would want to live with something on there mind forever remembering bad a bad event.

You should think about other people before you get in your car. Is being on a phone more import than your life and others lives?

Every year approximately a